In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [34]:
path = r'D:\NUS\project\LTA_data\raw_data\bus\2021'
lis_file = os.listdir(path=path)
lis_file

['origin_destination_bus_202101.csv',
 'origin_destination_bus_202102.csv',
 'origin_destination_bus_202103.csv',
 'origin_destination_bus_202104.csv',
 'origin_destination_bus_202105.csv',
 'origin_destination_bus_202106.csv',
 'origin_destination_bus_202107.csv',
 'origin_destination_bus_202108.csv',
 'origin_destination_bus_202109.csv',
 'origin_destination_bus_202110.csv',
 'origin_destination_bus_202111.csv',
 'origin_destination_bus_202112.csv']

In [3]:
bus_od = pd.read_csv(path+'\\'+'origin_destination_bus_202101.csv')
print('The number of bus records is: {}'.format(len(bus_od)))

The number of bus records is: 5329064


In [4]:
def get_raw_data(path,i):
    path = r'D:\NUS\project\LTA_data\raw_data\bus\2020'
    lis_file = os.listdir(path=path)
    
    df = pd.read_csv(path+'\\'+lis_file[i])
    
    return df

In [5]:
subzone = gpd.read_file(r"D:\NUS\project\Singapore_subzone\subzone_2019\MasterPlan_SUBZONE_2019\master-plan-2019-subzone-boundary-no-sea-geojson-polygon.shp")
subzone_n = np.unique(list(subzone['SUBZONE_N']))
print(len(subzone_n))

332


In [6]:
bus_pts = gpd.read_file(r"D:\NUS\project\LTA_data\point_data\bus_stops\bus_stops_wszone.shp")
bus_pts.PTcode = bus_pts.PTcode.astype(int)
bus_pts = bus_pts.sort_values('PTcode')
bus_pts.index = range(len(bus_pts))
bus_pts = bus_pts.drop_duplicates('PTcode')

bus_pts = bus_pts.to_crs('epsg:4326')
subzone = subzone.to_crs('epsg:4326')
bus_pts_sj = bus_pts.sjoin(subzone,how = 'left')

stops_szone = bus_pts_sj[['PTcode','SUBZONE_N_right']]
lis_bus_pts = list(bus_pts_sj['PTcode'])
lis_szone = list(bus_pts_sj['SUBZONE_N_right'])

In [9]:
lis1 = (list(set(lis_szone)))
lis1.pop(1)
lis2 = list(subzone_n)

for i in lis2:
    if not(i in lis1):
        print(i)

CAIRNHILL
CHANGI BAY
CONEY ISLAND
GARDEN
JURONG ISLAND AND BUKOM
LORONG HALUS NORTH
MARINA EAST
MOUNT EMILY
NORTH-EASTERN ISLANDS
ONE TREE HILL
PULAU PUNGGOL BARAT
PULAU PUNGGOL TIMOR
PULAU SELETAR
PUNGGOL CANAL
SEMAKAU
SIMPANG NORTH
SIMPANG SOUTH
SOUTHERN GROUP
SUDONG
TANJONG IRAU


In [10]:
bus_pts = bus_pts.drop_duplicates('PTcode')

In [11]:
len(np.unique(list(bus_pts.PTcode)))

5108

In [12]:
bus_stop_api = pd.read_csv(r"D:\NUS\project\LTA_data\point_data\bus_stops\busstop_from_api_202205.csv")
lis_bus_api_pts = list(set(bus_stop_api['BusStopCode']))
print('the number of bus stpos: {}'.format(len(lis_bus_api_pts)))

the number of bus stpos: 5067


In [13]:
lis_origin_pts = list(set(bus_od['ORIGIN_PT_CODE']))
len(lis_origin_pts)

4991

In [14]:
count = 0
lis_missing = []
for i in lis_origin_pts:
    if not(i in lis_bus_pts):lis_missing.append(i)
        
print(len(lis_missing))

0


In [15]:
def get_RawData(path,i):
    lis_file = os.listdir(path=path)
    
    df = pd.read_csv(path+'\\'+lis_file[i])
    
    return df

In [16]:
def aggreg_od(od):
    origin,destination,trips_num = 'ORIGIN_PT_CODE','DESTINATION_PT_CODE','TOTAL_TRIPS'
    df = od.groupby([origin,destination]).sum(trips_num).reset_index().drop('TIME_PER_HOUR',axis = 1)
    df['Origin_Subzone'] = None
    df['Destination_Subzone'] = None
    return df

In [17]:
def output_subzone(x,subzone,stops):
    index = stops.index(x)
    subzone_n = subzone[index]    
    return subzone_n
   
def origin_bus_stops_subzone_matching(subzone_n,aggregate_od,x):
    df = aggregate_od.loc[aggregate_od['ORIGIN_PT_CODE'] == x].copy()
    df.loc[:,'Origin_Subzone'] = subzone_n
    return df

def destination_bus_stops_subzone_matching(subzone_n,aggregate_od,x):
    df = aggregate_od.loc[aggregate_od['DESTINATION_PT_CODE'] == x].copy()
    df.loc[:,'Destination_Subzone'] = subzone_n
    return df

In [18]:
def get_origin_subzone(aggregate_od,subzone,bus_pts):
    cols = aggregate_od.columns
    tmp_origin = pd.DataFrame(columns = cols)
    for i in list(bus_pts):
        subzone_n = output_subzone(i,subzone,bus_pts)
        subzone_mrt_stop = origin_bus_stops_subzone_matching(subzone_n,aggregate_od,i)
        tmp_origin = tmp_origin.append(subzone_mrt_stop)
        del(subzone_n)
    return tmp_origin


def get_destination_subzone(aggregate_od,subzone,bus_pts):
    cols = aggregate_od.columns
    tmp_destination = pd.DataFrame(columns = cols)
    for i in list(bus_pts):
        subzone_n = output_subzone(i,subzone,bus_pts)
        subzone_mrt_stop = destination_bus_stops_subzone_matching(subzone_n,aggregate_od,i)
        tmp_destination = tmp_destination.append(subzone_mrt_stop)
        del(subzone_n)
    return tmp_destination

In [55]:
def main():
    raw_data = get_RawData(path,8)
    aggregate_train_od = aggreg_od(raw_data)
    print(lis_file[8])
    
    bus_origin_subzone = get_origin_subzone(aggregate_train_od, lis_szone,lis_bus_pts)
    bus_destination_subzone = get_destination_subzone(aggregate_train_od,lis_szone,lis_bus_pts)
    
    bus_origin_subzone['index'] = bus_origin_subzone.index
    bus_destination_subzone['index'] = bus_destination_subzone.index
    
    bus_origin_subzone = bus_origin_subzone.sort_values('index')
    bus_destination_subzone = bus_destination_subzone.sort_values('index')
    
    bus_origin_subzone['Destination_Subzone'] = bus_destination_subzone['Destination_Subzone']
    
    bus_od_subzone = bus_origin_subzone.copy()
    
    return bus_od_subzone

In [56]:
if __name__ == '__main__':
    bus_od_subzone = main()

origin_destination_bus_202109.csv


C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_origin = tmp_origin.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_origin = tmp_origin.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_origin = tmp_origin.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_origin = tmp_origin.append(subzone_mrt_stop)
C:\Users

C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_origin = tmp_origin.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_origin = tmp_origin.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_origin = tmp_origin.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_origin = tmp_origin.append(subzone_mrt_stop)
C:\Users

C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_origin = tmp_origin.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_origin = tmp_origin.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_origin = tmp_origin.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_origin = tmp_origin.append(subzone_mrt_stop)
C:\Users

C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_origin = tmp_origin.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_origin = tmp_origin.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_origin = tmp_origin.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_origin = tmp_origin.append(subzone_mrt_stop)
C:\Users

...

C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_destination = tmp_destination.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_destination = tmp_destination.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_destination = tmp_destination.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_destination = tmp_d

C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_destination = tmp_destination.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_destination = tmp_destination.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_destination = tmp_destination.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_destination = tmp_d

C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_destination = tmp_destination.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_destination = tmp_destination.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_destination = tmp_destination.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_destination = tmp_d

C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_destination = tmp_destination.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_destination = tmp_destination.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_destination = tmp_destination.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_destination = tmp_d

C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_destination = tmp_destination.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_destination = tmp_destination.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_destination = tmp_destination.append(subzone_mrt_stop)
C:\Users\YOLO\AppData\Local\Temp/ipykernel_4116/951447664.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp_destination = tmp_d

In [57]:
bus_od_subzone = bus_od_subzone.drop('index',axis = 1)

In [58]:
result_path = r'D:\NUS\project\LTA_data\od_comuting_flow\bus'

bus_od_subzone.to_csv(result_path +'\\'+'bus_commuting_flow_202109.csv',index = False)